In [65]:
import findspark
findspark.init('c:/spark')
from pyspark import SparkConf, SparkContext
import collections
import re
from pyspark.sql import SparkSession
from pyspark.sql import Row
import pandas as pd
import collections
from pyspark.ml.linalg import Vectors
from sklearn.cluster import MiniBatchKMeans, KMeans
import numpy as np

In [66]:
spark = SparkSession.builder.config("spark.sql.warehouse.dir", "C:/temp").appName("SparkSQL").getOrCreate()

In [67]:
#inputLines = spark.sparkContext.textFile("test1.csv")
inputLines = spark.sparkContext.textFile("train_full_parsed_clean3.1.csv")
#inputLines = spark.sparkContext.textFile("train_full_parsed_clean5.csv")

In [68]:
header=inputLines.filter(lambda l: "_id" in l)

In [69]:
header.collect()

['id,trip_duration,vendor_id,pickup_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,great_circle_distance,duration,speed,speed_great_circle,hour_count,passenger_count,pickup_hour,pickup_weekday,pickup_month,snow,holiday,distance,route_cooridnates']

In [70]:
linesnoheader = inputLines.subtract(header)

In [71]:
linesnoheader.take(1)

['id2187720,428,2,2016-02-03 03:50:49,-74.00108337402342,40.7391014099121,-73.98416137695312,40.72916030883789,1804.6226836023934,529.8,17.3885046728972,15.179069301328546,795,2,3,2,2,0,0,2067.3,"[[-74.001057, 40.739136], [-73.990717, 40.734794], [-73.991462, 40.731745], [-73.98444, 40.728783], [-73.984159, 40.729159]]"']

In [82]:
#coord=pd.read_csv('route_coord_count.csv')
coord=pd.read_csv('newcounter.csv')

kmeans = MiniBatchKMeans(n_clusters=100, batch_size=10000,random_state=1234).fit(coord[['longitude','latitude']])
coord.loc[:, 'cluster'] = kmeans.predict(coord[['longitude','latitude']])

In [73]:
cluster=coord['cluster'].tolist()

cluster_name=['C'+str(i) for i in range(100)]


In [39]:
def topcoordinates(line):
    coordinates=re.findall('(\[\-\d+\.\d+\,\s\d+\.\d+\])',line)
    line_cluster=[]
    speed_great_circle=line.split(',')[11]
    hour_count=line.split(',')[12]
    for i in coordinates:
        temp=re.split('\[|, |\]',i)
        long=float(temp[1])
        lat=float(temp[2])
        cluster_group=kmeans.predict([[long,lat]])
        line_cluster+=[cluster_group[0]]
        
    line_cluster=list(set(line_cluster))
    all_cluster=[ float(speed_great_circle) if i in line_cluster else np.nan for i in range(100)]
        
    
    
    

    return [int(hour_count)]+all_cluster

    

In [83]:
line= 'id2187720,428,2,2016-02-03 03:50:49,-74.00108337402342,40.7391014099121,-73.98416137695312,40.72916030883789,1804.6226836023934,529.8,17.3885046728972,15.179069301328546,795,2,3,2,2,0,0,2067.3,"[[-74.001057, 40.739136], [-73.990717, 40.734794], [-73.991462, 40.731745], [-73.98444, 40.728783], [-73.984159, 40.729159]]"'
coordinates=re.findall('(\[\-\d+\.\d+\,\s\d+\.\d+\])',line)
line_cluster=[]
speed_great_circle=line.split(',')[11]
hour_count=line.split(',')[12]
for i in coordinates:
        temp=re.split('\[|, |\]',i)
        long=float(temp[1])
        lat=float(temp[2])
        cluster_group=kmeans.predict([[long,lat]])
        line_cluster+=[cluster_group[0]]
        
line_cluster=list(set(line_cluster))
all_cluster=[ float(speed_great_circle) if i in line_cluster else np.nan for i in range(100)]



In [84]:
line_cluster

[90, 74, 39]

In [40]:
coord = linesnoheader.map(topcoordinates)

In [41]:
feature=['hour_count']
colNames=feature+cluster_name

In [42]:
cluster_name=['C'+str(i) for i in range(100)]

In [43]:
train = coord.toDF(colNames)

In [45]:
#train.repartition(40).write.parquet('parquet_cluster_40_files')
train.repartition(1).write.csv('speed_cluster')

In [46]:
train=pd.read_csv('speed_cluster.csv',names=colNames)

In [47]:
train.loc[train.hour_count==795 , 'C40']

24188     NaN
91615     NaN
94352     NaN
104120    NaN
108097    NaN
125212    NaN
145020    NaN
145080    NaN
212678    NaN
242480    NaN
251979    NaN
255074    NaN
294639    NaN
303726    NaN
342591    NaN
392934    NaN
394577    NaN
403635    NaN
436261    NaN
450891    NaN
529559    NaN
636174    NaN
683498    NaN
724865    NaN
851287    NaN
853583    NaN
900161    NaN
911814    NaN
918261    NaN
991878    NaN
1041174   NaN
1153390   NaN
1159574   NaN
1177202   NaN
1196635   NaN
1276059   NaN
1329760   NaN
1373090   NaN
1380796   NaN
1440398   NaN
Name: C40, dtype: float64

In [16]:
(23.600927+20.549816)/2

22.0753715

In [60]:
pd.set_option('max_columns', 200)
train.head(2)

,id,trip_duration,vendor_id,pickup_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,great_circle_distance,duration,speed,speed_great_circle,hour_count,passenger_count,pickup_hour,pickup_weekday,pickup_month,snow,holiday,distance,route_cooridnates
0,id2875421,455,2,2016-03-14 17:24:55,-73.982155,40.767937,-73.964630,40.765602,1498.942981,421.6,15.889055,11.859769,1769,1,17,0,3,0,0,2008.2,"[[-73.982316, 40.767869], [-73.982163, 40.7677..."
1,id2377394,663,1,2016-06-12 00:43:35,-73.980415,40.738564,-73.999481,40.731152,1806.015862,587.0,13.653394,9.806421,3912,1,0,6,6,0,0,2514.5,"[[-73.980421, 40.738566], [-73.984941, 40.7323..."


In [48]:
df=train.groupby('hour_count').median()

In [50]:
df.to_csv('hour_sopeed_cluster.csv')

In [53]:
train=pd.read_csv('train_full_parsed_clean3.1.csv')

In [56]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1457392 entries, 0 to 1457391
Data columns (total 21 columns):
id                       1457392 non-null object
trip_duration            1457392 non-null int64
vendor_id                1457392 non-null int64
pickup_datetime          1457392 non-null object
pickup_longitude         1457392 non-null float64
pickup_latitude          1457392 non-null float64
dropoff_longitude        1457392 non-null float64
dropoff_latitude         1457392 non-null float64
great_circle_distance    1457392 non-null float64
duration                 1457392 non-null float64
speed                    1457392 non-null float64
speed_great_circle       1457392 non-null float64
hour_count               1457392 non-null int64
passenger_count          1457392 non-null int64
pickup_hour              1457392 non-null int64
pickup_weekday           1457392 non-null int64
pickup_month             1457392 non-null int64
snow                     1457392 non-null int64
holid

In [64]:
train.loc[(train.pickup_longitude==-74.00108337402342) & (train.dropoff_longitude==-73.98416137695312)].to_csv("test1.csv",index=False)